In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
# Setări generale
img_size = 48
data_dir = 'data'  # Directorul cu imagini
emotion_labels = {
    'anger': 0,
    'contempt': 1,
    'disgust': 2,
    'fear': 3,
    'happy': 4,
    'neutral': 5,
    'sad': 6,
    'surprise': 7
}

X, y, geom_features = [], [], []

def extract_geom_features(landmarks):
    features = []
    # Exemplu: Distanță între punctele disponibile
    features.append(np.linalg.norm(np.array(landmarks[0]) - np.array(landmarks[1])))
    features.append(np.linalg.norm(np.array(landmarks[2]) - np.array(landmarks[3])))
    return features


# Încărcarea imaginilor
for emotion, label in emotion_labels.items():
    folder_path = os.path.join(data_dir, emotion)
    if os.path.exists(folder_path):
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img_resized = cv2.resize(img_gray, (img_size, img_size))
            img_normalized = img_resized / 255.0
            X.append(np.expand_dims(img_normalized, axis=-1))
            y.append(label)
            # Simulare de caracteristici geometrice (poți înlocui cu landmark-uri reale)
            geom_features.append(extract_geom_features([(30, 30), (40, 40), (50, 50), (60, 60)]))

X = np.array(X, dtype='float32')
y = np.array(y)
geom_features = np.array(geom_features, dtype='float32')

# One-hot encoding pentru etichete
y = to_categorical(y, num_classes=len(emotion_labels))


In [6]:
X_train, X_temp, y_train, y_temp, geom_train, geom_temp = train_test_split(X, y, geom_features, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test, geom_val, geom_test = train_test_split(X_temp, y_temp, geom_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
geom_train = scaler.fit_transform(geom_train)
geom_val = scaler.transform(geom_val)
geom_test = scaler.transform(geom_test)


In [7]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator()

def generator_with_geom(gen, geom_data):
    while True:
        img_batch, y_batch = next(gen)
        indices = gen.index_array[:len(img_batch)]
        geom_batch = geom_data[indices]
        yield [img_batch, geom_batch], y_batch

train_gen = generator_with_geom(train_datagen.flow(X_train, y_train, batch_size=64, shuffle=True), geom_train)
val_gen = ([X_val, geom_val], y_val)


In [8]:
image_input = Input(shape=(img_size, img_size, 1), name='image_input')
geom_input = Input(shape=(geom_train.shape[1],), name='geom_input')

# Bloc convoluțional
x = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)

# Procesarea caracteristicilor geometrice
y = Dense(32, activation='relu')(geom_input)

# Concatenare și straturi dense
combined = concatenate([x, y])
z = Dense(64, activation='relu')(combined)
z = Dropout(0.5)(z)
output = Dense(len(emotion_labels), activation='softmax')(z)

model = Model(inputs=[image_input, geom_input], outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 48, 48, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 48, 48,    │        320 │ image_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 24, 24,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 24, 24,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 12, 12,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ geom_input          │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 9216)      │          0 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │         96 │ geom_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 9248)      │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │    591,936 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8)         │        520 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 611,368 (2.33 MB)

 Trainable params: 611,368 (2.33 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(X_train) // 64,
    epochs=30,
    validation_data=val_gen
)


TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.